In [ ]:
import requests
import random
from bs4 import BeautifulSoup


In [ ]:
url = 'https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0061.TW'
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "accept-encoding": "gzip, deflate, br", 
    "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7", 
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "sec-gpc": "1",
    "referer": "https://www.google.com/",
    "dnt": "1", 
    "upgrade-Insecure-Requests": "1", 
    "user-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Mobile Safari/537.36", 
}



In [ ]:
# test requests but no use
res = requests.get(url, headers=headers).content
soup = BeautifulSoup(res, 'html.parser')
# soup

In [ ]:
import selenium
selenium.__version__
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from time import sleep
import numpy as np
from tqdm import tqdm


In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'
chrome_path = 'C:/Users/Administrator/.wdm/drivers/chromedriver/win32/96.0.4664.45/chromedriver.exe'
url = 'https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid=2824.HK'
opt = webdriver.ChromeOptions()
opt.add_argument('--user-agent=%s' % user_agent)


### ETFs
- https://www.moneydj.com/etf/x/Basic/Basic0007.xdjhtm?etfid=SPY

In [ ]:
import json

In [ ]:
# catch all etf company names
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

driver.get(url)
sleep(0.5)

etf_list = [item.text for item in driver.find_elements_by_xpath('//select[@id="Menu1"]//option')]
print(etf_list)

driver.quit()



In [ ]:
etf_list_asia = etf_list[212:]


In [ ]:
# catch all etf of each companies
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

etf_options = dict()
driver.get(url)
for etf in tqdm(etf_list_asia):
    
#     idx = etf_list.index(etf)
#     print(idx)

    # get current company's etf list
    option_list = [item.text for item in driver.find_elements_by_xpath('//select[@id="Menu2"]//option')]
    etf_options.update({etf: option_list})
    
    # go to another company
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, 
                                        '//span[@id="DivMenu1"]'))
    ).click()
    action = ActionChains(driver)
    action.send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    
    sleep(np.random.randint(1, 5))


driver.quit()




In [ ]:
# catch all stock (element ratio) of each etf of each companies
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)

etf_element = dict()
for etf in tqdm(etf_options.keys()):
    for etfid in etf_options[etf]:
        
        etfid = etfid.split('〈')[1][:-1]
        url = f'https://www.moneydj.com/ETF/X/Basic/Basic0007.xdjhtm?etfid={etfid}'
        driver.get(url)
        
        try:
            table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, 
                                                '//table[@id="ctl00_ctl00_MainContent_MainContent_stable3"]')))
            table_content = table.text
        except:
            print(url)
            table_content = ' '
#         driver.find_element_by_xpath('//table[@id="ctl00_ctl00_MainContent_MainContent_stable3"]').text
        etf_element.update({etfid: table_content})
        
        sleep(np.random.randint(1, 5))

driver.quit()
        

In [ ]:
# reformat the table content to list of tuples
etf_element_new = dict()
for etfid in etf_element.keys():
    
    if etf_element[etfid] == ' ':
        etf_element_new.update({etfid: ' '})
    else:
        list_table = [line.split(' ') for line in etf_element[etfid].split('\n')]
        etf_element_new.update({etfid: [(row[0], row[1], row[2]) for row in list_table[1:]]})



In [ ]:
# write to json
with open('etf_element_ratio.json', 'w', encoding='utf-8') as f:
    json.dump(etf_element_new, f, ensure_ascii=False, indent=4)
    

In [ ]:
# load from json
with open('etf_element_ratio.json', 'r', encoding='utf-8') as f:
    etf_element_test = json.load(f)
    

In [ ]:
# A mapping dict between ETF id & name
etf_id_name_mapping = dict()

for comp in etf_options.keys():
    for etf in etf_options[comp]:
        etf_id = etf.split('〈')[-1][:-1]
        etf_name = etf.split('〈')[0]
        etf_id_name_mapping.update({etf_id: etf_name})
        
# etf_id_name_mapping


### 基金
- https://www.moneydj.com/funddj/yb/YP301000.djhtm

In [ ]:
import re
import pandas as pd


In [ ]:
url = 'https://www.moneydj.com/funddj/yb/YP301000.djhtm'
account = 'abaoyang'
password = 'YGyang615thebest'


In [ ]:
# catch type of funds
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get(url)
sleep(1)
fund_types = [(item.text, item.get_attribute('href')) for item 
              in driver.find_elements_by_xpath('//div[@class="InternalSearch"]//a')]
print(fund_types[:5])

fund_company_list = list()
for item in tqdm(fund_types):
    driver.get(item[1])
    fund_company = [(item.text, item.get_attribute('href')) for item 
                         in driver.find_elements_by_xpath('//table[@id="oMainTable"]//td/a') if len(item.text.strip()) > 0]
    print(fund_company[:6])
    fund_list = fund_company[0::2]
    print(fund_list[:3])
    company_list = fund_company[1::2]
    company_list = [item[0] for item in company_list]
    print(company_list[:3])
    for i in range(len(fund_list)):
        fund_company_list += [(company_list[i], fund_list[i][0], fund_list[i][1])]
    
    sleep(np.random.randint(3, 7))
    
    
driver.quit()




In [ ]:
fund_company_dict = {'list': fund_company_list}

# store to json file
with open('fund_company_dict.json', 'w', encoding='utf-8') as f:
    json.dump(fund_company_dict, f, ensure_ascii=False, indent=4)


In [ ]:
# load from json
with open('../../KG-try/basic_info_collector/from_moneyDJ/fund_company_dict.json', 'r', encoding='utf-8') as f:
    fund_company_dict = json.load(f)
fund_company_list = fund_company_dict['list']


In [16]:
# catch ratio of funds
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
fund_url = 'https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACFP72'
driver.get(fund_url)
sleep(15)


fund_ratio = dict()

for fund in tqdm(fund_company_list):
    

    fund_name = fund[1]
    fund_id = fund[2].split('a=')[-1]
    fund_ratio.update({fund_id: dict()})
    fund_ratio[fund_id].update({'基金名稱': fund_name})

    info_url = 'https://www.moneydj.com/funddj/yp/yp011000.djhtm?a=%s' % (fund_id)
    fund_url = 'https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=%s' % (fund_id)
    
    # 持股成分
    driver.get(fund_url)
    try:
        ratio_content = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, 
                                                    '//table[@class="t04"]//table[@class="t01"]//td[contains(@class, "t3")]')))
        ratio_content = [item.text for item 
                         in driver.find_elements_by_xpath('//table[@class="t04"]//table[@class="t01"]//td[contains(@class, "t3")]')]
        ratio_content = np.array(ratio_content).reshape(len(ratio_content)//4, 4).tolist()
        
        update_date = driver.find_element_by_xpath('//table[@class="t04"]//td[@class="t3n1c1"]').text
#         print(update_date, ratio_content)
    except:
        ratio_content = ' '
        update_date = ' '
        print(fund_url)

    finally:
        fund_ratio[fund_id].update({'更新時間': update_date})
        fund_ratio[fund_id].update({'持股': ratio_content})
        
    # 基本資料
    driver.get(info_url)
    try:
        info_col = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//table[@class="t04"]//td[@class="t2c1"]')))
        info_col = [item.text for item
                    in driver.find_elements_by_xpath('//table[@class="t04"]//td[@class="t2" or @class="t2c1"]')]
        info_val = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//table[@class="t04"]//td[@class="t3t2"]')))
        info_val = [item.text for item 
                    in driver.find_elements_by_xpath('//table[@class="t04"]//td[@class="t3t2"]')]
        
    except Exception as e:
        print(e)
        info_col, info_val = list(), list()
        
    finally:
        res = dict(zip(info_col, info_val))
        fund_ratio[fund_id].update({'基本資料': res})
#         print(res)


    sleep(np.random.randint(3, 7))
    
driver.quit()


# store to json file
with open('../../KG-try/basic_info_collector/from_moneyDJ/20220117/fund_holdings_final.json', 'w', encoding='utf-8') as f:
    json.dump(fund_ratio, f, ensure_ascii=False, indent=4)



  0%|▏                                                                             | 6/3672 [01:38<15:58:09, 15.68s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACCA98


  0%|▏                                                                             | 7/3672 [01:51<14:56:22, 14.67s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACTR12


  0%|▏                                                                             | 8/3672 [01:59<13:06:50, 12.88s/it]

...

100%|██████████████████████████████████████████████████████████████████████████▉| 3670/3672 [10:02:00<00:24, 12.26s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACJS79


100%|██████████████████████████████████████████████████████████████████████████▉| 3671/3672 [10:02:11<00:11, 11.90s/it]

https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACFH64


100%|███████████████████████████████████████████████████████████████████████████| 3672/3672 [10:02:21<00:00,  9.84s/it]


In [ ]:
fund_ratio_copy['ACDD01']

In [ ]:
# catch ratio of funds
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
fund_id = 'ACDS99'
fund_url = 'https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=%s' % (fund_id)
info_url = 'https://www.moneydj.com/funddj/yp/yp011000.djhtm?a=%s' % (fund_id)

# 基本資料
driver.get(info_url)

# try:
info_col = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//table[@class="t04"]//td[@class="t2c1"]')))
info_col = [item.text for item
            in driver.find_elements_by_xpath('//table[@class="t04"]//td[@class="t2" or @class="t2c1"]')]
info_val = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//table[@class="t04"]//td[@class="t3t2"]')))
info_val = [item.text for item 
            in driver.find_elements_by_xpath('//table[@class="t04"]//td[@class="t3t2"]')]

# except Exception as e:
#     print(e)
#     info_col, info_val = list(), list()

# finally:
res = dict(zip(info_col, info_val))
print(res)


sleep(np.random.randint(3, 7))

driver.quit()



In [14]:
password
# fund_ratio.keys()
# fund_ratio['ACDS99']


'YGyang615thebest'

In [15]:
account 
# np.array(ratio_content).reshape(72//4, 4)

'abaoyang'

In [ ]:
# store to json file
with open('fund_element_ratio_tables.json', 'w', encoding='utf-8') as f:
    json.dump(fund_ratio, f, ensure_ascii=False, indent=4)
    
with open('fund_id_name_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(fund_id_name_mapping, f, ensure_ascii=False, indent=4)


In [ ]:
# 只存股票項目不存比例
fund_ratio_simple = dict()
for k in fund_ratio.keys():
    
    update_date = fund_ratio[k].split(' 更新。')[0]
    content = ' '.join(fund_ratio[k].split('\n')[3:])
    cl = np.unique([char.strip() for char in re.findall(r'[\u4e00-\u9fff- a-zA-Z]+|91APP', content) 
                    if (char.strip() != '') and (char.strip() != '-') 
                    and (char.strip() != 'KY') and (char.strip() != '-KY')]).tolist()
    
    fund_ratio_simple.update({k: {'更新時間': update_date, '持股': cl}})


# store to json file
with open('fund_holdings.json', 'w', encoding='utf-8') as f:
    json.dump(fund_ratio_simple, f, ensure_ascii=False, indent=4)


In [ ]:
fund_ratio_simple['ACAI174']


In [ ]:
# 轉存 3 元組為 csv
collect = list()
for k in tqdm(fund_ratio_simple.keys()):
    for item in fund_ratio_simple[k]['持股']:
        collect += [(k, '持有', item)]

len(collect)
pd.DataFrame(collect, columns=['fund_id', 'rel', 'target_name']).drop_duplicates().to_csv('fund_holding_relation.csv', index=False)


### 公司基本資訊
- MoneyDJ: https://concords.moneydj.com/z/zc/zca/zca.djhtm?a=2303
    - 有些網頁是亂碼、有些會有空格問題抓起來比較麻煩
    
- Goodinfo: https://goodinfo.tw/StockInfo/BasicInfo.asp?STOCK_ID=2303


In [ ]:
url = 'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a=2303'

list_stock_id = pd.read_csv('listed_company.csv', header=None)
list_stock_id.columns = ['comp']
list_stock_id = list_stock_id['comp'].str.split('　', expand=True)
list_stock_id.columns = ['id', 'name']
stock_ids = list_stock_id.id.unique()
stock_ids


In [ ]:
url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
res = requests.get(url, headers=headers).content
soup = BeautifulSoup(res, 'html.parser')
keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
try:
    idx = keys.index('公司地址')
    _dict = dict(zip(keys[:idx+1], values[:idx+1]))

except:
    print(f'broken id: {stock_id}')

finally:
    print(_dict)
    

In [ ]:
def getCompanyTable(url):
    print(url)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
    values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
    try:
        idx = keys.index('公司地址')
        _dict = dict(zip(keys[:idx+1], values[:idx+1]))
    except:
        print(f'broken ')
        print(keys, values)
        _dict = ''
    return _dict




import chardet
chardet.detect('¶}½L»ù'.encode('utf-8'))


In [ ]:
# for stock_id in broken_ids:
stock_id = 1324
url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
getCompanyTable(url)


In [ ]:
# comp_dict = dict()
# broken_ids = list()

selected_stock_ids = ['1236','1307','1321','1419','1439','1470','1541','1592','1614','1727','1805','2028','2115','2115','2211','2250','2388','2390','2404','2405','2424','2430','2433','2438','2468','2501','2501','2543','2712','2753','2910','2936','2945','3021','3035','3051','3059','3138','3266','3419','3515','3532','3701','3714','4190','4426','4440','4555','4560','4755','4930','4994','5222','5225','5284','5546','5608','6136','6172','6189','6201','6209','6224','6230','6289','6405','6442','6515','6715','6743','6754','6756','6768','6770','6781','6790','6806','8104','8427','8467','8478','8482','8488','9926','9927']

for stock_id in tqdm(selected_stock_ids):

    url = f'https://concords.moneydj.com/z/zc/zca/zca.djhtm?a={stock_id}'
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    keys = [item.text for item in soup.find_all('td', {'class': 't4t1'})]
    values = [item.text for item in soup.find_all('td', {'class': ['t3n1', 't3t1']}) if item.text.strip() != '']
    try:
        idx = keys.index('公司地址')
        _dict = dict(zip(keys[:idx+1], values[:idx+1]))
        comp_dict.update({stock_id: _dict})
    
    except:
        print(f'broken id: {stock_id}')
        broken_ids.append(stock_id)
        continue
    
    finally:
        sleep(np.random.randint(3, 8))



In [ ]:
for stock_id in tqdm(selected_stock_ids):
    print(comp_dict[stock_id])
    

In [ ]:
selected_stock_ids[2]

In [ ]:
# write to json
with open('company_basic_info.json', 'w', encoding='utf-8') as f:
    json.dump(comp_dict, f, ensure_ascii=False, indent=4)
    

#### 類股分類
- 上市
- 上櫃
- 興櫃
- 電子產業
- 概念股
- 集團股


In [ ]:
def getStockCates(idx):
    
    market_categories = ['上市','上櫃','興櫃','電子產業','概念股','集團股']
    market_cat = market_categories[idx]
    url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT={market_cat}'
    headers['User-Agent'] = random.choice(user_agent_list)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    list_stock_cate = soup.find('table', {'id': f'MENU{idx+2}'}).find_all('td', {'colspan': '4'})
    list_stock_cate = [re.findall(r'[\u4e00-\u9fff]+', str(item)) for item in list_stock_cate]
    list_stock_cate = [item[-1] for item in list_stock_cate if len(item) >0 and market_cat not in item[-1]]
    return list_stock_cate


list_stock_cate = getStockCates(idx=1)
list_stock_cate


In [ ]:
industries = list_stock_cate.copy()
url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT=上市&INDUSTRY_CAT={list_stock_cate[0]}'


In [ ]:
def getCateStockList(market_cat, ind):
    
    url = f'https://goodinfo.tw/StockInfo/StockList.asp?MARKET_CAT={market_cat}&INDUSTRY_CAT={ind}'
    headers['User-Agent'] = random.choice(user_agent_list)
    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res, 'html.parser')
    list_stock = [item.text for item in soup.find('table', {'class': 'r10_0_0_10 b1 p4_1'}).find_all('td')]
    list_stock = [(x, y) for x, y in zip(list_stock[::2], list_stock[1::2]) if x != '代號']
    return list_stock



In [ ]:
market_categories = ['上市','上櫃','興櫃','電子產業','概念股','集團股']
stock_categories = dict()
sleep_time = 10

for idx in tqdm(range(len(market_categories))):
    _ind_stocks = dict()
    market_cat = market_categories[idx]
    industries = getStockCates(idx=idx)
    print(industries)
    
    sleep(np.random.randint(5, 8))
    
    for ind in industries:
        try:
            list_stock = getCateStockList(market_cat=market_cat, ind=ind)
        except:
            print(url)
            # 發生錯誤時 多休息幾秒緩衝 避免被網站擋
            sleep_time += 5
            continue
            
        finally:
            _ind_stocks.update({ind: list_stock})
            print(ind, _ind_stocks[ind], '\n')

        sleep(np.random.randint(sleep_time, sleep_time+5))
        
    stock_categories.update({market_cat: _ind_stocks})
    print(market_cat, stock_categories[market_cat])



In [ ]:
stock_categories.keys()
stock_categories['上櫃']['保險業']


In [ ]:
with open('stock_categories.json', 'w', encoding='utf-8') as f:
    json.dump(stock_categories, f, ensure_ascii=False, indent=4)


In [ ]:
url = f'https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID={stock_id}'


#### SemEval 2016 Task5 ABSA datasets
- xml parser

In [ ]:
from xml.etree import ElementTree
import xmltodict

# trial data
url = 'https://alt.qcri.org/semeval2016/task5/data/uploads/trial-data/mobilephones_trial_chinese.xml'

res = requests.get(url, headers=headers).content
# tree = ElementTree.fromstring(res)
res_dict = xmltodict.parse(res)
res_dict


In [ ]:
res_dict.keys()
res_dict['Reviews'].keys()
len(res_dict['Reviews']['Review'])
res_dict['Reviews']['Review'][0].keys()
res_dict['Reviews']['Review'][0]['sentences'].keys()
len(res_dict['Reviews']['Review'][0]['sentences']['sentence'])
res_dict['Reviews']['Review'][0]['sentences']['sentence'][1].keys()
res_dict['Reviews']['Review'][0]['sentences']['sentence'][1]['Opinions']['Opinion']['@category']


#### Other

In [ ]:
# check table data
# 英文公司名中間有空格，和分隔符衝突

empty = list()
diff = list()
valid = list()
for k in fund_ratio.keys():
    
    cl = list()
    for row in fund_ratio[k].split('\n'):
        cl.append([char.strip() for char in row.split(' ') if len(char.strip()) > 0])
    
    try: 
        check = len(cl[3])
        if check not in [6, 8]:
            diff.append(k)
        else:
            valid.append(k)
    except:
        empty.append(k)


empty
diff

In [ ]:
# test the login pop-up
driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get('https://www.moneydj.com/funddj/yl/yp013000.djhtm?a=ACKY39')
sleep(1)

# 在網頁外面的彈出 selenium 沒辦法操作
#action = ActionChains(driver)
#action.send_keys(account).send_keys(Keys.TAB).send_keys(password).send_keys(Keys.ENTER).perform()


In [ ]:
driver.quit()

In [ ]:
cl = list()
for row in fund_ratio[diff[11]].split('\n'):
    cl.append([char.strip() for char in row.split(' ') if len(char.strip()) > 0])

cl
# [row.split(' ') for row in fund_ratio['AC0001'].split('\n')]
diff[11]

In [ ]:
k = 'ACAI175'

content = ' '.join(fund_ratio[k].split('\n')[3:])
cl = np.unique([char.strip() for char in re.findall(r'[\u4e00-\u9fff- a-zA-Z]+|91APP', content) 
                if (char.strip() != '') and (char.strip() != '-')]).tolist()

content

In [ ]:
len(fund_ratio_simple.keys())

In [ ]:
fund_ratio[k]

In [ ]:
url = 'https://www.tej.com.tw/webtej/doc/uid.htm'

driver = webdriver.Chrome(executable_path=chrome_path, options=opt)
driver.set_window_size(1024, 850)
driver.get(url)
text = [item.text for item in driver.find_elements_by_xpath('//font')]
sleep(1)
driver.quit()

In [ ]:
end = text.index('上櫃公司代碼一覽表')
start = text.index('1101   台泥')
stock_comp = [item.strip() for item in text[start:end] if (item.strip() != '') and re.findall('[0-9]+', item)]
stock_names = [item.split(' ')[-1].strip() for item in stock_comp]
stock_ids = [item.split(' ')[0].strip() for item in stock_comp]


In [ ]:
re.findall('[0-9]+', '1110   東泥')

In [ ]:
df_stock_comp = pd.DataFrame([stock_ids, stock_names]).T
df_stock_comp.columns = ['id', 'name']
df_stock_comp.to_csv('df_stock_comp.csv', index=False)

In [ ]:
df_stock_comp